In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=5,
    leverages={"APTUSDT-PERP.BINANCE": 5}
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="DEBUG"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, OrderAccepted, OrderFilled
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 10
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)

    
    def on_order_accepted(self, event: OrderAccepted):
        if self.__limit_order is not None:
            if self.__limit_order.client_order_id == event.client_order_id:
                self.log.info(f"After limit order accepted with qty {self.__limit_order.quantity} balances locked: " + 
                              f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.MAGENTA)
                return
                
        self.log.info(f"After unidentified order accepted balances locked: " + 
            f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.MAGENTA)

    
    def on_order_filled(self, event: OrderFilled):
        self.log.info(f"After filled qty {event.last_qty} balances locked: " + 
             f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.CYAN)

    

    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry(tick)
        self.check_for_exit()


    def check_for_entry(self, tick: TradeTick):
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )
        
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    
    def on_position_opened(self, event: PositionOpened):
        self.position = self.cache.position(event.position_id)

        if self.position.side == PositionSide.LONG:
            order_side = OrderSide.BUY

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 - 0.001)
            )
            
        else:
            order_side = OrderSide.SELL

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 + 0.001)
            )

        
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )

        self.__limit_order = self.order_factory.limit(
            self.config.instrument_id,
            order_side,
            quantity,
            limit_price,
            reduce_only=False,
            post_only=False,
            time_in_force=TimeInForce.GTC,
            expire_time=None,
            quote_quantity=False,
            emulation_trigger=TriggerType.NO_TRIGGER,
            trigger_instrument_id=None,
            exec_algorithm_id=None,
            exec_algorithm_params=None,
            display_qty=None
        )
        self.submit_order(self.__limit_order)

    
    def on_position_changed(self, event: PositionChanged):
        self.log.info(f"After position changed to amount {self.position.quantity} balances locked: " + 
             f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.CYAN)

    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-13T19:31:36.383916837Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-13T19:31:36.383939240Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-13T19:31:36.383941034Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-13T19:31:36.383942010Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-13T19:31:36.383942848Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-13T19:31:36.383943951Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-13T19:31:36.383944629Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-13T19:31:36.383946360Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-13T19:31:36.383947256Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-04-13T19:31

In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

x_close=0.0, realized_return=0.00000, realized_pnl=-0.05958642 USDT, unrealized_pnl=0.00084000 USDT, ts_opened=1733788811884000000, ts_last=1733788811884000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:11.884000000Z [INFO] BACKTESTER-001.MACDStrategy: After position changed to amount 10.0 balances locked: 10.3407141
2024-12-10T00:00:11.884000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000011-001-000-33') with VenueOrderId('BINANCE-1-032')
2024-12-10T00:00:11.884000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=10.94971020 USDT
2024-12-10T00:00:11.884000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=98.63231430 USDT, locked=11.55749250 USDT, free=87.07482180 USDT)], margins=[MarginBalance(initial=10.94971020 USDT, maintenance=0.60778230 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=ea6b25d7-9ee5-4653-9f

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,exec_spawn_id,tags,init_id,ts_init,ts_last,price,expire_time_ns,is_post_only,display_qty,trigger_instrument_id
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,10.0,...,None,None,e45e5423-b945-4e68-96c1-ad122b76c83b,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000008-001-000-3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-003,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,10.0,...,None,None,09b099af-1934-4b4e-9cc1-3d9c75f02429,2024-12-10 00:00:08.205000+00:00,2024-12-10 00:00:08.205000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000008-001-000-5,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-004,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-006,MARKET,BUY,10.0,...,None,None,853ef5c3-e097-42a4-b7a3-b50d64e6ee16,2024-12-10 00:00:08.218000+00:00,2024-12-10 00:00:08.218000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000008-001-000-6,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-005,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-012,LIMIT,BUY,10.0,...,None,None,6bcfd1f2-77a5-40fc-852a-10e5effeb3b8,2024-12-10 00:00:08.218000+00:00,2024-12-10 00:00:10.099000+00:00,11.91620,NaN,False,NaN,NaN
O-20241210-000009-001-000-7,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-006,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-008,MARKET,SELL,10.0,...,None,None,371013a6-f169-4796-bec2-c5624e378d6b,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241210-000108-001-000-397,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-378,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-489,MARKET,SELL,10.0,...,None,None,8d3d61bf-39a2-4c5e-9532-912c0efeea2b,2024-12-10 00:01:08.732000+00:00,2024-12-10 00:01:08.732000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000108-001-000-398,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-379,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-491,MARKET,SELL,10.0,...,None,None,6d09b529-ed7f-4075-94b4-ec6b01a76233,2024-12-10 00:01:08.732000+00:00,2024-12-10 00:01:08.732000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000108-001-000-400,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-381,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-492,MARKET,BUY,10.0,...,None,None,e99e79f8-ebc0-45d8-bcee-cdd1a6d95fba,2024-12-10 00:01:08.842000+00:00,2024-12-10 00:01:08.842000+00:00,NaN,NaN,NaN,NaN,NaN


ESTER-001.MACDStrategy: <--[EVT] PositionClosed(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000011-001-000-34, closing_order_id=O-20241210-000012-001-000-36, entry=SELL, side=FLAT, signed_qty=0.0, quantity=0.0, peak_qty=10.0, currency=USDT, avg_px_open=11.915997, avg_px_close=11.919391, realized_return=-0.00028, realized_pnl=-0.15311695 USDT, unrealized_pnl=0.00000000 USDT, ts_opened=1733788811884000000, ts_last=1733788812479000000, ts_closed=1733788812479000000, duration_ns=595000000)
2024-12-10T00:00:12.482000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91960,0.9,BUYER,245141486,1733788812482000000)
2024-12-10T00:00:12.482000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000012-001-000-37, side=BUY, type=MARKET, quantity=10.0, time_in_force=GTC, post_o

In [10]:
engine.trader.generate_positions_report()

strument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000014-001-000-39, account_id=BINANCE-001, ts_event=1733788814483000000)
2024-12-10T00:00:14.483000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Market: bid=42.2 @ 11.92320, ask=42.2 @ 11.92320, last=11.92320
2024-12-10T00:00:14.483000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Applying fills to MarketOrder(SELL 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=SUBMITTED, client_order_id=O-20241210-000014-001-000-39, venue_order_id=None, position_id=None, tags=None), venue_position_id=None, position=Position(LONG 10.0 APTUSDT-PERP.BINANCE, id=APTUSDT-PERP.BINANCE-MACDStrategy-000), fills=[(Price(11.92320), Quantity(10.0))]
2024-12-10T00:00:14.483000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000014-001-000-39') with VenueOrderId('BINANCE-1-038')
2024-12-10T00:00:14.483000000Z [DEBUG] BACKTESTER-001.Portfolio: Calculated PnLs: [Money(0.03509000, USDT)]
2024-12-10T00:00:14.483000000

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000-7d9b5f09-f00c-49ce-bd39-eba563dc16f2,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,O-20241210-000008-001-000-3,SELL,FLAT,0.0,10.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788808205000000,2024-12-10 00:00:08.204999936+00:00,3.157000e+09,11.935610,11.928018,[0.11931814 USDT],0.00064,-0.04339814 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-77477b38-12a5-4c29-9330-c0683d61fc13,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000008-001-000-5,O-20241210-000009-001-000-7,BUY,FLAT,0.0,10.0,1733788808218000000,2024-12-10 00:00:08.218000+00:00,1733788809609000000,2024-12-10 00:00:09.608999936+00:00,1.391000e+09,11.928191,11.929914,[0.11929053 USDT],0.00014,-0.10206053 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-21ab413c-674e-43b4-b47f-982efc04b7f1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000009-001-000-8,O-20241210-000008-001-000-6,SELL,FLAT,0.0,10.0,1733788809609000000,2024-12-10 00:00:09.609000+00:00,1733788810099000000,2024-12-10 00:00:10.099000064+00:00,4.900000e+08,11.929809,11.916200,[0.08348145 USDT],0.00114,0.05260855 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-5cf963f0-074c-4311-94a2-f08f9f5e7891,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-10,O-20241210-000010-001-000-13,SELL,FLAT,0.0,20.0,1733788810099000000,2024-12-10 00:00:10.099000+00:00,1733788810101000000,2024-12-10 00:00:10.100999936+00:00,2.000000e+06,11.921950,11.930600,[0.20274180 USDT],-0.00073,-0.37574180 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-a349095a-5056-4653-af79-531f738d5f0e,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-14,O-20241210-000010-001-000-16,BUY,FLAT,0.0,10.0,1733788810101000000,2024-12-10 00:00:10.101000+00:00,1733788810163000000,2024-12-10 00:00:10.163000064+00:00,6.200000e+07,11.928200,11.920700,[0.11924450 USDT],-0.00063,-0.19424450 USDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APTUSDT-PERP.BINANCE-MACDStrategy-000-bff09d59-bb5d-4689-8ca6-4b2b7c41cfba,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000108-001-000-389,O-20241210-000108-001-000-391,BUY,FLAT,0.0,10.0,1733788868375000000,2024-12-10 00:01:08.375000+00:00,1733788868471000000,2024-12-10 00:01:08.471000064+00:00,9.600000e+07,11.906300,11.904100,[0.11905200 USDT],-0.00018,-0.14105200 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-4247439a-c765-473d-bd71-199f871a5feb,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000108-001-000-392,O-20241210-000108-001-000-394,SELL,FLAT,0.0,10.0,1733788868471000000,2024-12-10 00:01:08.471000+00:00,1733788868732000000,2024-12-10 00:01:08.732000+00:00,2.610000e+08,11.904000,11.905100,[0.11904550 USDT],-0.00009,-0.13004550 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-9df5423d-915b-4208-92b3-1f4beb775405,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000108-001-000-395,O-20241210-000108-001-000-397,BUY,FLAT,0.0,10.0,1733788868732000000,2024-12-10 00:01:08.732000+00:00,1733788868732000000,2024-12-10 00:01:08.732000+00:00,NaN,11.904600,11.903500,[0.11904050 USDT],-0.00009,-0.13004050 USDT


, client_order_id=O-20241210-000015-001-000-43, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:00:15.148000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000015-001-000-43, account_id=BINANCE-001, ts_event=1733788815148000000)
2024-12-10T00:00:15.148000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Market: bid=50.4 @ 11.92180, ask=50.4 @ 11.92180, last=11.92180
2024-12-10T00:00:15.148000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Applying fills to MarketOrder(BUY 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=SUBMITTED, client_order_id=O-20241210-000015-001-000-43, venue_order_id=None, position_id=None, tags=None), venue_position_id=None, position=None, fills=[(Price(11.92180), Quantity(10.0))]
2024-12-10T00:00:15.148000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000015-001-000-43') with VenueOrderId('BINANCE-1-042')
2024-1

In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

 OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000015-001-000-47, account_id=BINANCE-001, ts_event=1733788815752000000)
2024-12-10T00:00:15.752000000Z [DEBUG] BACKTESTER-001.Portfolio: Calculated PnLs: []
2024-12-10T00:00:15.752000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=14.59767900 USDT
2024-12-10T00:00:15.752000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=98.06126073 USDT, locked=14.65237987 USDT, free=83.40888086 USDT)], margins=[MarginBalance(initial=14.59767900 USDT, maintenance=0.05470087 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=1dca2906-98ba-4d7a-a5d0-bea80424303f)
2024-12-10T00:00:15.752000000Z [DEBUG] BACKTESTER-001.Portfolio: Updated OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000015-001-000-46, venue_order_id=BINANCE-1-045, account_id=BINANCE-00

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,99.94032195,0.06087207,99.87944988,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
2024-12-10 00:00:05.048000+00:00,99.94032195,1.82737080,98.11295115,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
2024-12-10 00:00:08.205000+00:00,99.95381915,1.82737080,98.12644835,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 00:01:08.732000+00:00,84.37558217,83.27001540,1.10556677,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
2024-12-10 00:01:08.842000+00:00,84.26618417,83.27001540,0.99616877,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
2024-12-10 00:01:08.856000+00:00,84.26082566,82.66298280,1.59784286,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}
2024-12-10 00:01:08.856000+00:00,84.20664471,82.71763960,1.48900511,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '82.6625...",False,{}


trategy-000, order_side=BUY, order_type=LIMIT, last_qty=0.9, last_px=11.90970 USDT, commission=0.00214375 USDT, liquidity_side=MAKER, ts_event=1733788817966000000)
2024-12-10T00:00:17.966000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000010-001-000-21, venue_order_id=BINANCE-1-020, account_id=BINANCE-001, trade_id=BINANCE-1-055, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=BUY, order_type=LIMIT, last_qty=0.9, last_px=11.90970 USDT, commission=0.00214375 USDT, liquidity_side=MAKER, ts_event=1733788817966000000)
2024-12-10T00:00:17.966000000Z [INFO] BACKTESTER-001.MACDStrategy: After filled qty 0.9 balances locked: 16.423071
2024-12-10T00:00:17.966000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=-9.1
2024-12-10T00:00:17.966000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=0.55273382 USDT
2024-12-10T00:00:17.966000000Z [INFO] BACKTESTER-001.M